In [1]:
#cleaning the data

In [2]:
import re
import pandas as pd
import numpy as np
import requests
import os
import json
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
from IPython.core.display import HTML
from datetime import date, datetime


In [3]:
senators_file= "1976-2018-senate.csv"

In [4]:
# new_senate=pd.read_csv(senators_file)

new_senate = pd.read_csv(senators_file, encoding= 'unicode_escape')


In [5]:
df= pd.DataFrame(new_senate)
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
0,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Sam Steiger,republican,False,total,321236,741210,False,20171011.0
1,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Wm. Mathews Feighan,independent,False,total,1565,741210,False,20171011.0
2,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Dennis DeConcini,democrat,False,total,400334,741210,False,20171011.0
3,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Allan Norwitz,libertarian,False,total,7310,741210,False,20171011.0
4,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Bob Field,independent,False,total,10765,741210,False,20171011.0


In [6]:
datayear = df[(df.year>=2014) & (df.year<=2018)]
datayear

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
2861,2014,Alabama,AL,1,63,41,US Senate,statewide,gen,False,NaN,NaN,True,total,22484,818090,False,20171011.0
2862,2014,Alabama,AL,1,63,41,US Senate,statewide,gen,False,Jeff Sessions,republican,False,total,795606,818090,False,20171011.0
2863,2014,Alaska,AK,2,94,81,US Senate,statewide,gen,False,NaN,NaN,True,total,1376,282400,False,20171011.0
2864,2014,Alaska,AK,2,94,81,US Senate,statewide,gen,False,Mark Begich,democrat,False,total,129431,282400,False,20171011.0
2865,2014,Alaska,AK,2,94,81,US Senate,statewide,gen,False,Mark S. Fish,libertarian,False,total,10512,282400,False,20171011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,55,35,25,US Senate,statewide,gen,False,NaN,NaN,True,total,42,2657841,False,20190110.0
3417,2018,Wyoming,WY,56,83,68,US Senate,statewide,gen,False,John Barrasso,republican,False,total,136210,203420,False,20190110.0
3418,2018,Wyoming,WY,56,83,68,US Senate,statewide,gen,False,Gary Trauner,democrat,False,total,61227,203420,False,20190110.0
3419,2018,Wyoming,WY,56,83,68,US Senate,statewide,gen,False,Joseph Porambo,libertarian,False,total,5658,203420,False,20190110.0


In [7]:
new_df=datayear[["year","state","state_po","candidate","party","candidatevotes","writein","totalvotes"]]
new_df

,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,NaN,NaN,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,NaN,NaN,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,NaN,NaN,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [8]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 2861 to 3420
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            560 non-null    int64 
 1   state           560 non-null    object
 2   state_po        560 non-null    object
 3   candidate       468 non-null    object
 4   party           439 non-null    object
 5   candidatevotes  560 non-null    int64 
 6   writein         560 non-null    bool  
 7   totalvotes      560 non-null    int64 
dtypes: bool(1), int64(3), object(4)
memory usage: 35.5+ KB


In [9]:
new_df.columns

Index(['year', 'state', 'state_po', 'candidate', 'party', 'candidatevotes',
       'writein', 'totalvotes'],
      dtype='object')

In [10]:
new_df['candidate'] = new_df['candidate'].replace(np.NaN, "Write in")
new_df


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,NaN,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,Write in,NaN,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,NaN,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [11]:
new_df

,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,NaN,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,Write in,NaN,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,NaN,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [12]:
new_df['party'] = new_df['party'].replace(np.NaN, "No Party")
new_df


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [13]:
new_df["party"]= new_df["party"].replace(["republican"], "Republican")
new_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,Republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [14]:
new_df["party"]= new_df["party"].replace(["democrat"], "Democrat")
new_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,Democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,Republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,Democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,libertarian,5658,False,203420


In [15]:
new_df["party"]= new_df["party"].replace(["libertarian"], "Libertarian")
new_df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,state,state_po,candidate,party,candidatevotes,writein,totalvotes
2861,2014,Alabama,AL,Write in,No Party,22484,True,818090
2862,2014,Alabama,AL,Jeff Sessions,Republican,795606,False,818090
2863,2014,Alaska,AK,Write in,No Party,1376,True,282400
2864,2014,Alaska,AK,Mark Begich,Democrat,129431,False,282400
2865,2014,Alaska,AK,Mark S. Fish,Libertarian,10512,False,282400
...,...,...,...,...,...,...,...,...
3416,2018,Wisconsin,WI,Write in,No Party,42,True,2657841
3417,2018,Wyoming,WY,John Barrasso,Republican,136210,False,203420
3418,2018,Wyoming,WY,Gary Trauner,Democrat,61227,False,203420
3419,2018,Wyoming,WY,Joseph Porambo,Libertarian,5658,False,203420


In [16]:
new_df.shape

(560, 8)

In [17]:
new_df.to_csv('votersinfo.csv', index = False)